# L2: Language modelling

In this lab you will implement and train two neural language models: the fixed-window model mentioned in Lecture&nbsp;2.3, and the recurrent neural network model from Lecture&nbsp;2.5. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from google.colab import drive

For this lab, you should use the GPU if you have one:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data

The data for this lab is [WikiText](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the set of &lsquo;Good&rsquo; and &lsquo;Featured&rsquo; articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the &lsquo;training&rsquo; and the &lsquo;validation&rsquo; section of the data. We fill this container by reading the corresponding text files. The only processing that we do is to whitespace-tokenize, and to replace each newline character with a special token `<eos>` (end-of-sentence).

In [ ]:
class WikiText(object):

    def __init__(self):
        self.vocab = {}
        self.train = self.read_data('wiki.train.tokens')
        self.valid = self.read_data('wiki.valid.tokens')

    def read_data(self, path):
        ids = []
        with open(path) as source:
            for line in source:
                for token in line.split() + ['<eos>']:
                    if token not in self.vocab:
                        self.vocab[token] = len(self.vocab)
                    ids.append(self.vocab[token])
        return ids

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [ ]:
wikitext = WikiText()

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.vocab))

Tokens in train: 2088628
Tokens in valid: 217646
Vocabulary size: 33278


## Problem 1: Fixed-window neural language model

In this section you will implement and train the fixed-window neural language model proposed by [Bengio et al. (2003)](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) and introduced in Lecture&nbsp;2.3. Recall that an input to the network takes the form of a vector of $n-1$ integers representing the preceding words. Each integer is mapped to a vector via an embedding layer. (All positions share the same embedding.) The embedding vectors are then concatenated and sent through a two-layer feed-forward network with a non-linearity in the form of a rectified linear unit (ReLU) and a final softmax layer.

### Problem 1.1: Vectorize the data

Your first task is to write code for transforming the data in the WikiText container into a vectorized form that can be fed to the fixed-window model. Complete the skeleton code in the cell below:

In [ ]:
def vectorize_fixed_window(wikitext_data, n):
    valid_x = [wikitext_data[i:i+n-1] for i in range(len(wikitext_data)) if i < len(wikitext_data)-n]
    valid_y = [wikitext_data[i+n-1] for i in range(len(wikitext_data)) if i < len(wikitext_data)-n]

    valid_x = torch.tensor(valid_x)
    valid_y = torch.tensor(valid_y)
    return valid_x, valid_y

Your function should meet the following specification:

**vectorize_fixed_window** (*wikitext_data*, *n*)

> Transforms WikiText data (a list of word ids) into a pair of tensors $\mathbf{X}$, $\mathbf{y}$ that can be used to train the fixed-window model. Let $N$ be the total number of $n$-grams from the token list; then $\mathbf{X}$ is a matrix with shape $(N, n-1)$ and $\mathbf{y}$ is a vector with length $N$.

⚠️ Your function should be able to handle arbitrary values of $n \geq 1$.

#### 🤞 Test your code

Test your implementation by running the code in the next cell. Does the output match your expectation?

In [ ]:
valid_x, valid_y = vectorize_fixed_window(wikitext.train, 3)
print(valid_x.size())

torch.Size([2088625, 2])


### Problem 1.2: Implement the model

Your next task is to implement the fixed-window model based on the graphical specification given in the lecture.

In [ ]:
class FixedWindowModel(torch.nn.Module):

    def __init__(self, n, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim, device=device)
       # self.embedding.weight.data.uniform_(0.0, 0.0)

        self.embedding_dim = embedding_dim
        self.n = n
        self.hiddenLinear = nn.Linear((n-1) * embedding_dim, hidden_dim, bias = True, device=device)
        self.softmaxLinnear = nn.Linear(hidden_dim, n_words, bias = True, device=device)

    def forward(self, x):
        #Create embedding layer
        x = self.embedding(x)
        #Concatinate vector
        x = x.view(-1, (self.n-1) * self.embedding_dim)
        print(x.shape,(self.n-1) * self.embedding_dim )
        x = self.hiddenLinear(x)
        x = torch.relu(x)
        x = self.softmaxLinnear(x)
        return x

NameError: ignored

Here is the specification of the two methods:

**__init__** (*self*, *n*, *n_words*, *embedding_dim*=50, *hidden_dim*=50)

> Creates a new fixed-window neural language model. The argument *n* specifies the model&rsquo;s $n$-gram order. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the hidden layer of the feedforward network, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, n-1)$, where $B$ is the batch size. The output of the forward pass is a tensor of shape $(B, V)$ where $V$ is the number of words in the vocabulary.

**Hint:** The most efficient way to implement the vector concatenation in this model is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 1.3: Train the model

Your final task is to write code to train the fixed-window model using minibatch gradient descent and the cross-entropy loss function.

For your convenience, the following cell contains a utility function that randomly samples minibatches of a specified size from a pair of tensors:

In [ ]:
def batchify(x, y, batch_size):
    random_indices = torch.randperm(len(x))
    for i in range(0, len(x) - batch_size + 1, batch_size):
        indices = random_indices[i:i+batch_size]
        yield x[indices].to(device), y[indices].to(device)
    remainder = len(x) % batch_size
    if remainder:
        indices = random_indices[-remainder:]
        yield x[indices].to(device), y[indices].to(device)

def countPerplexit(valid_x, valid_y,n, model,batch_size):
  final_list = []
  for x,y in batchify(valid_x, valid_y, batch_size):
    with torch.no_grad():
      output = model.forward(x)
      loss = F.cross_entropy(output,y)
      perplexity = torch.exp(loss)
      final_list.append(perplexity)
  meanPerplexity = sum(final_list)/len(final_list)
  return meanPerplexity



What remains to be done is the implementation of the training loop. This should be a straightforward generalization of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [ ]:
def train_fixed_window(n, n_epochs=1, batch_size=3200, lr=1e-2):
    # Initialize the model
    model = FixedWindowModel(n, n_words = len(wikitext.vocab), embedding_dim=50, hidden_dim=50)

    # Initialize the optimizer. Here we use Adam rather than plain SGD
    optimizer = optim.Adam(model.parameters(), lr=lr)
    valid_x, valid_y = vectorize_fixed_window(wikitext.train, n)

    for epoch in range(n_epochs):
        model.train()
        for x,y in batchify(valid_x, valid_y, batch_size):
            optimizer.zero_grad()
            output = model.forward(x)
            loss = F.cross_entropy(output,y)
            loss.backward()
            optimizer.step()
        valid_x, valid_y = vectorize_fixed_window(wikitext.valid, n)
        perplexity = countPerplexit(valid_x, valid_y,n,model,batch_size)
        print(perplexity)

    return model

model_fixed_window = train_fixed_window(2, n_epochs=1)

tensor(317.8192, device='cuda:0')


Here is the specification of the training function:

**train_fixed_window** (*n*, *n_epochs* = 1, *batch_size* = 3200, *lr* = 0.01)

> Trains a fixed-window neural language model of order *n* using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

The code in the cell below trains a bigram model.

In [ ]:
model_fixed_window = train_fixed_window(2, n_epochs=1)

tensor(330.6390, device='cuda:0')


**⚠️ Your submitted notebook must contain output demonstrating a validation perplexity of at most 350.**

**Hint:** Computing the validation perplexity in one go may exhaust your computer&rsquo;s memory and/or take a lot of time. If you run into this problem, break the computation down into minibatches and take the average perplexity.

#### 🤞 Test your code

To see whether your network is learning something, print the loss and/or the perplexity on the training data. If the two values are not decreasing over time, try to find the problem before wasting time (and energy) on useless training.

Training and even evaluation will take some time – on a CPU, you should expect several minutes per epoch, depending on hardware. To speed things up, you can train using a GPU; our reference implementation runs in less than 30 seconds per epoch on [Colab](http://colab.research.google.com).

## Problem 2: Recurrent neural network language model

In this section you will implement the recurrent neural network language model that was presented in Lecture&nbsp;2.5. Recall that an input to the network is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer, from which we read off the index of the word at position $i+1$. In theory, the input vector could represent the complete training data or at least a complete sentence; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*, the **backpropagation-through-time horizon**.

### Problem 2.1: Vectorize the data

As in the previous problem, your first task is to transform the data in the WikiText container into a vectorized form that can be fed to the model.

In [ ]:
def vectorize_rnn(wikitext_data, bptt_len):
    wikitext_data = torch.tensor(wikitext_data, device=device)

    valid_x = torch.zeros((math.floor(len(wikitext_data)/bptt_len), bptt_len), device=device)
    valid_y = torch.zeros((math.floor(len(wikitext_data)/bptt_len), bptt_len), device=device)
    index = 0
    for i in range(0,len(wikitext_data),bptt_len):
      if len(wikitext_data[i:-1]) < bptt_len:
        break
      else:
        valid_x[index] =  wikitext_data[i:i+bptt_len]
        valid_y[index] = wikitext_data[i+1:i+bptt_len+1]
        index += 1
    valid_x = valid_x.long()
    valid_y = valid_y.long()
    return valid_x, valid_y


Your function should meet the following specification:

**vectorize_rnn** (*wikitext_data*, *bptt_len*)

> Transforms a list of token indexes into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [ ]:
valid_x, valid_y = vectorize_rnn(wikitext.train, 32)


torch.Size([65269, 32]) torch.Size([65269, 32]) 2088628 32


### Problem 2.2: Implement the model

Your next task is to implement the recurrent neural network model based on the graphical specification given in the lecture.

In [ ]:
class RNNModel(nn.Module):

    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim, device=device)
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim, bias = True, device=device, batch_first=True)
        self.linear = nn.Linear(hidden_dim, n_words, device=device)

    def forward(self, x):
        x = self.embedding(x)
        output, (h_n, c_n) = self.LSTM(x)
        x = self.linear(output)

        return x


Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 50, *hidden_dim* = 50)

> Creates a new recurrent neural network language model. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 2.3: Train the model

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [ ]:
  def countPerplexitRNN(valid_x, valid_y, model, batch_size):
    final_list = []
    index=0
    for x,y in batchify(valid_x, valid_y, batch_size):
      with torch.no_grad():
        output = model.forward(x)
        output = output.view(-1, len(wikitext.vocab))
        y = y.view(-1)
        loss = F.cross_entropy(output,y)
        perplexity = torch.exp(loss)
        final_list.append(perplexity)
    meanPerplexity = sum(final_list)/len(final_list)
    return meanPerplexity


  def train_rnn(n_epochs=1, batch_size=100, bptt_len=32, lr=0.01):
      model = RNNModel(len(wikitext.vocab))
      #should be wikitext.train
      train_x, train_y = vectorize_rnn(wikitext.train, bptt_len)

      optimizer = optim.Adam(model.parameters(), lr=lr)
      for epoch in range(n_epochs):
          model.train()
          for x,y in batchify(train_x, train_y, batch_size):
              optimizer.zero_grad()
              output = model.forward(x)
              output = output.view(-1, len(wikitext.vocab))
              y = y.view(-1)
              loss = F.cross_entropy(output,y)
              loss.backward()
              optimizer.step()

          # Should be wikitext.valid
          valid_x, valid_y = vectorize_rnn(wikitext.valid, bptt_len)
          perplexity = countPerplexitRNN(valid_x, valid_y,model,batch_size)
          print(perplexity)

      return model



Here is the specification of the training function:

**train_rnn** (*n_epochs* = 1, *batch_size* = 100, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [ ]:
model_rnn = train_rnn(n_epochs=1, batch_size=100)
#model = RNNModel(len(wikitext.vocab))
#bptt_len=32
#batch_size=100
#valid_x, valid_y = vectorize_rnn(wikitext.valid, bptt_len)
#perplexity = countPerplexitRNN(valid_x, valid_y,model,batch_size)
#print(perplexity)

tensor(310.4526, device='cuda:0')


**⚠️ Your submitted notebook must contain output demonstrating a validation perplexity of at most 310.**

## Problem 3: Parameter initialization (reflection)

Since the error surfaces that gradient search explores when training neural networks can be very complex, it is important to choose &lsquo;good&rsquo; initial values for the parameters. In PyTorch, the weights of the embedding layer are initialized by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the standard deviation and/or the distribution affects the perplexity of your feed-forward language model. Write a short report about your experience (ca. 150 words). Use the following prompts:

* What different settings for the initialization did you try? What results did you get?
* How can you choose a good initialization strategy?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

For the Fixed window model and RNN we tested 7 different settings and got the following results.

Fixed_window \\
Without changing the weights, 344 \\
[0.0;0.0], 317 \\
[0;.2.0],  330 \\
[1.0;.2.0], 337 \\
[-0.5;0.5],  317 \\
[-1.0;1.0],  326 \\
[0.0;50.0], 1046 \\
[0;.100.0],  960 \\

---
Rnn  \\
Without changing the weights, 301 \\
[0.0;0.0], 310 \\
[0;.2.0],  560 \\
[1.0;.2.0], 697 \\
[-0.5;0.5],  299 \\
[-1.0;1.0],  299 \\
[0.0;50.0], 1063 \\
[0;.100.0],  1039 \\



What we could see that for both models it was a better choice of sampling with smaller numbers. While for high weights the perplexity also got really high.

In this lab we have got a deeper understanding about how the training the model works, this was learned by reading more about it and also implementing it several times.